In [ ]:
import pandas as pd
import sys
sys.path.append('.')
from Extract_function.get_video_data import get_video_data

In [7]:
video_ids = [
    "nsPdOoooRw8",
    "ifnNAskSguU",
    "Ee6bJEJcNFY"
]

In [ ]:
# Initialize an empty list to store the data for each video.
# Each element in this list will be a dictionary containing details for one video.
rows = []

# Iterate over the list of video IDs.
# For each video ID, fetch its data using the get_video_data function.
for vid in video_ids:
    # Call the get_video_data function (defined in API/get_video_data.py)
    # to retrieve snippet and statistics for the current video ID (vid).
    # The function returns a dictionary with video details or None if an error occurs.
    row = get_video_data(vid)
    
    # Check if the get_video_data function returned a valid row (i.e., not None).
    # This ensures that only successfully fetched video data is processed.
    if row:
        # If data was successfully fetched (row is not None),
        # append the dictionary containing video details to the 'rows' list.
        rows.append(row)


Number of items returned: 1

Number of items returned: 1

Number of items returned: 1


In [9]:
df = pd.DataFrame(rows)

In [18]:
# Convert the 'publishedAt' column to datetime objects.
# This is necessary for performing date/time calculations, such as determining the age of the video.
df["publishedAt"] = pd.to_datetime(df["publishedAt"])

# Get the current timestamp in UTC.
# Using UTC ensures consistency, especially if the 'publishedAt' times are also in UTC or can be converted to it.
now_utc = pd.Timestamp.now(tz="UTC")

# Calculate the age of each video in days.
# This is done by subtracting the 'publishedAt' timestamp from the current UTC timestamp.
# The `.dt.days` accessor then extracts the difference in full days.
df["age_in_days"] = (now_utc - df["publishedAt"]).dt.days

# Calculate the engagement rate for each video.
# Engagement rate is defined here as the sum of likes and comments,
# divided by the total view count, then multiplied by 100 to express it as a percentage.
# This metric can help gauge how much interaction a video receives relative to its views.
df["engagementRate"] = (df["likeCount"] + df["commentCount"]) / df["viewCount"] * 100

In [19]:
df.head(3)

,videoId,title,publishedAt,viewCount,likeCount,commentCount,age_in_days,engagementRate
0,nsPdOoooRw8,Cuidemos las tetas,2023-10-19 12:50:09+00:00,7043,108,1,569,1.547636
1,ifnNAskSguU,Universal - Añade una Silla esta Navidad,2023-11-21 04:00:23+00:00,2306715,124,0,536,0.005376
2,Ee6bJEJcNFY,Oriental - La parranda Sí Puedo,2023-12-09 01:40:25+00:00,881112,85,3,518,0.009987


In [ ]:
# Import necessary libraries for interacting with Google Sheets and Google Drive
import gspread  # For interacting with Google Sheets API
from gspread_dataframe import set_with_dataframe  # For easily writing pandas DataFrames to Google Sheets
from google.oauth2.service_account import Credentials  # For authenticating using a service account

# 1. Define the OAuth scopes and load credentials
# SCOPES define the permissions the script will have.
# "https://www.googleapis.com/auth/spreadsheets" allows reading and writing to Google Sheets.
# "https://www.googleapis.com/auth/drive" allows managing files in Google Drive (needed by gspread to find/create sheets).
SCOPES = ["https://www.googleapis.com/auth/spreadsheets",
          "https://www.googleapis.com/auth/drive"
          ]
# Load credentials from the service account JSON file.
# This file contains the private key and other details needed to authenticate as the service account.
# The 'scopes' parameter specifies the permissions requested.
creds = Credentials.from_service_account_file("service_account.json", scopes=SCOPES)
# Authorize the gspread client with the loaded credentials.
# This creates a client object that can be used to interact with Google Sheets.
client = gspread.authorize(creds)

# 2. Open the Google Sheet
# Open the spreadsheet titled "Video Pulse Data".
# This assumes the service account has access to this spreadsheet.
# If the sheet doesn't exist and the service account has Drive creation permissions, it might create one,
# but typically you'd ensure the sheet exists and is shared with the service account's email.
spreadsheet = client.open("Video Pulse Data")
# Select the first worksheet in the spreadsheet (often named 'Sheet1' by default).
sheet1      = spreadsheet.sheet1

# 3. Clear any old data from the selected worksheet (optional step)
# This ensures that the sheet only contains the new data being written.
# If you want to append data or update specific ranges, you would skip or modify this step.
sheet1.clear()

# 4. Dump your DataFrame into the selected worksheet
# The `set_with_dataframe` function from `gspread_dataframe` writes the entire pandas DataFrame `df`
# to the specified worksheet `sheet1`, starting from cell A1 by default.
# It includes headers and index based on its default behavior (which can be customized).
set_with_dataframe(sheet1, df)

# Print a confirmation message to the console indicating success.
print("DataFrame successfully written to Google Sheet!")


✅ DataFrame successfully written to Google Sheet!
